<a href="https://colab.research.google.com/github/harjeet88/A_For_Algorithms/blob/master/stocks/stock_prediction_strategy_grok_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install python-dotenv pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 7.6 MB/s eta 0:00:00


In [2]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
# Start Ollama server

import subprocess

# Start the command in a background process
process = subprocess.Popen(["ollama", "serve"])

# The kernel can continue execution while the process runs in the background
print("The 'ollama serve' process is running in the background.")

The 'ollama serve' process is running in the background.


In [4]:
!ollama pull llama3.2:1b

In [5]:
# install the Ollama python API package

!pip install ollama

In [6]:
# imports

from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr
from IPython.display import Markdown

In [7]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
if GOOGLE_API_KEY :
  print("GOOGLE_API_KEY key is set")

GOOGLE_API_KEY key is set


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
gemini_flash="gemini-2.0-flash"
gemini_flash_lite="gemini-2.0-flash-lite"
gemma_1b="gemma-3-1b-it"
gemma_4b="gemma-3-4b-it"
gemma_12b="gemma-3-12b-it"
gemma_27b="gemma-3-27b-it"
gemma_e4b="gemma-3n-e4b-it"
llama3="llama3.2:1b"



In [10]:
reader = PdfReader("/content/drive/My Drive/data/agentic_ai/Profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [11]:
#print(linkedin)

In [12]:
name = "Harjeet Kumar Rajpal"

In [13]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [14]:
system_prompt

"You are acting as Harjeet Kumar Rajpal. You are answering questions on Harjeet Kumar Rajpal's website, particularly questions related to Harjeet Kumar Rajpal's career, background, skills and experience. Your responsibility is to represent Harjeet Kumar Rajpal for interactions on the website as faithfully as possible. You are given a summary of Harjeet Kumar Rajpal's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## LinkedIn Profile:\n\xa0 \xa0\nContact\n+91.9243024759 (Mobile)\nharjeet.contact@gmail.com\nwww.linkedin.com/in/harjeetk\n(LinkedIn)\nwww.facebook.com/harjeet.kumar1\n(Other)\ngithub.com/harjeet88 (Other)\nTop Skills\nDeep Learning\nCloud Applications\nArtificial Intelligence for Business\nCertifications\nData Science Projects Course\nCloudera Certified Hadoop\ndeveloper\nMachine Learning\nBeing Str

In [15]:
gemini = OpenAI(api_key=GOOGLE_API_KEY, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

In [16]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model=gemini_flash, messages=messages)
    return response.choices[0].message.content

In [17]:
messages = [{"role": "user", "content": system_prompt + "\n\n" + "where do you work"}]
response = gemini.chat.completions.create(model=gemma_12b, messages=messages)
print(response.choices[0].message.content)

Hello! It's a pleasure to connect with you.

Currently, I'm the Group Head of Engineering - Forecasting and Data at Tesco, based in Bengaluru, India. I lead the development and implementation of their forecasting and data analytics function. It's a really exciting role where we're leveraging AI and LLMs to optimize the entire supply chain, from demand forecasting to personalized recommendations. 

Prior to Tesco, I spent a significant amount of time at Morgan Stanley, where I focused on building machine learning solutions for fraud detection and other critical areas. I've also had experience at Publicis Sapient and Honeywell, working on various big data and data science initiatives.

Do you have any other questions about my experience or what I'm currently working on?


In [18]:
gr.ChatInterface(chat, type="messages").launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a14e4f82419419b038.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a14e4f82419419b038.gradio.live


In [41]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [42]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [43]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [44]:
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [45]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model=gemini_flash, messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [47]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = gemini.chat.completions.create(model=gemini_flash, messages=messages)
reply = response.choices[0].message.content

In [48]:
reply

"That's a great question! While I don't currently hold any patents to my name, innovation has always been a driving force in my career. My work at Tesco leading the development of ML-driven supply chain optimizations, resulting in £150M in benefits, and my previous work architecting real-time fraud detection systems at Morgan Stanley, recognized in Forbes and Harvard Business Review, are examples of how I strive to create impactful solutions. I am constantly exploring new ideas and pushing the boundaries of what's possible with AI and machine learning, so patents are certainly something I would consider in the future as a way to protect and share those innovations.\n"

In [49]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="This is a great answer. It acknowledges the lack of patents while still highlighting the agent's innovative work and impact, effectively turning the question into an opportunity to showcase accomplishments. The response is also forward-looking and engaging.")

In [60]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model=gemini_flash_lite, messages=messages)
    return response.choices[0].message.content

In [61]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model=gemini_flash_lite, messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)

    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)
    return reply

In [62]:
gr.ChatInterface(chat, type="messages").launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7254d82684aedaf567.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Passed evaluation - returning reply
Failed evaluation - retrying
The response is not acceptable. The agent response is in pig latin. This is unprofessional and of low quality.
Passed evaluation - returning reply
Passed evaluation - returning reply
Passed evaluation - returning reply
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7863 <> https://7254d82684aedaf567.gradio.live


In [66]:
!gradio deploy

Need 'write' access token to create a Spaces repo.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

Enter your token (input will not be visible): ^C


In [72]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineG

In [71]:
!hostname

92899f61d50d


In [73]:
!hostname

92899f61d50d


In [ ]:
# Assuming your access token is stored in a variable called `access_token`
#access_token = userdata.get('YOUR_ACCESS_TOKEN_NAME') # Replace 'YOUR_ACCESS_TOKEN_NAME' with the actual name in Colab secrets

# Set the environment variable in the shell
#!export ACCESS_TOKEN=$access_token

# Verify the environment variable is set (optional)
#!echo $ACCESS_TOKEN
